In [ ]:
# This notebook is for combining data gathered with the Steam API and from a downloaded Kaggle dataset

In [1]:
import pandas as pd
import re

In [ ]:
# All reviews
df_steamAPI = pd.read_csv('reviews_from_json.csv')
df_kaggle = pd.read_csv('steam_reviews.csv')

In [ ]:
# Elden Ring: 1245620
# Dark Souls, Remaster: 570940
# Dark Souls 2: 236430
# Dark Souls 2, Scholar: 335300
# Dark Souls 3: 374320
# Sekiro GOTY: 814380
fromSoftfiles = [
    'review_1245620.json','review_570940.json','review_236430.json',
    'review_335300.json','review_374320.json','review_814380.json'
]

df = pd.DataFrame()
for file in fromSoftfiles:
    temp_df = pd.read_json(file)
    df = pd.concat([df, temp_df], ignore_index=True)

In [9]:
# Only keep rows where 'reviews' is a dictionary
df = df[df['reviews'].apply(lambda x: isinstance(x, dict))]

# Normalize the 'reviews' column
expanded_reviews_df = pd.json_normalize(df['reviews'])

# Concatenate it with your original dataframe if necessary
df = pd.concat([df, expanded_reviews_df], axis=1)

# Drop the original 'reviews' column as it is no longer needed
df = df.drop(columns=['reviews'])

# Save your dataframe to a CSV
df.to_csv('FromSoft_reviews.csv')

In [27]:
df_steamAPI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338568 entries, 0 to 338567
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      338568 non-null  int64  
 1   query_summary                   0 non-null       float64
 2   cursors                         0 non-null       float64
 3   recommendationid                338526 non-null  float64
 4   language                        338526 non-null  object 
 5   review                          337686 non-null  object 
 6   timestamp_created               338526 non-null  float64
 7   timestamp_updated               338526 non-null  float64
 8   voted_up                        338526 non-null  object 
 9   votes_up                        338526 non-null  float64
 10  votes_funny                     338526 non-null  float64
 11  weighted_vote_score             338526 non-null  float64
 12  comment_count   

In [28]:
df_steamAPI.head()

,Unnamed: 0,query_summary,cursors,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,...,steam_china_location,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,timestamp_dev_responded,developer_response
0,0,NaN,NaN,163751705.0,english,Addicted\n,1.714284e+09,1.714284e+09,True,0.0,...,NaN,7.656120e+16,0.0,9.0,1850.0,1850.0,1850.0,1.714284e+09,NaN,NaN
1,1,NaN,NaN,163721470.0,english,great game,1.714246e+09,1.714246e+09,True,0.0,...,NaN,7.656120e+16,0.0,3.0,13013.0,5575.0,13013.0,1.714246e+09,NaN,NaN
2,2,NaN,NaN,163627781.0,english,Fun Co-op game,1.714146e+09,1.714146e+09,True,0.0,...,NaN,7.656120e+16,0.0,1.0,2061.0,2061.0,465.0,1.714311e+09,NaN,NaN
3,3,NaN,NaN,163623793.0,schinese,单人魂游，多人爽游,1.714142e+09,1.714142e+09,True,0.0,...,NaN,7.656120e+16,214.0,135.0,92.0,0.0,92.0,1.712632e+09,NaN,NaN
4,4,NaN,NaN,163618609.0,koreana,가격대 이상의 재미.\r\n늦게 알아서 패치는 이제 없겠지 ㅜ 갓겜,1.714137e+09,1.714137e+09,True,0.0,...,NaN,7.656120e+16,0.0,1.0,2743.0,0.0,2743.0,1.702132e+09,NaN,NaN


In [29]:
df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434891 entries, 0 to 434890
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   date_posted             434891 non-null  object
 1   funny                   434891 non-null  int64 
 2   helpful                 434891 non-null  int64 
 3   hour_played             434891 non-null  int64 
 4   is_early_access_review  434891 non-null  bool  
 5   recommendation          434891 non-null  object
 6   review                  433375 non-null  object
 7   title                   434891 non-null  object
dtypes: bool(1), int64(3), object(4)
memory usage: 23.6+ MB


In [30]:
df_kaggle.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title
0,2019-02-10,2,4,578,False,Recommended,&gt Played as German Reich&gt Declare war on B...,Expansion - Hearts of Iron IV: Man the Guns
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight


In [31]:
#Drop Nulls
df_steamAPI.dropna(subset=['review'], inplace=True)
df_kaggle.dropna(subset=['review'], inplace=True)

#Keep only English reviews from the API dataset
df_steamAPI = df_steamAPI[df_steamAPI['language'] == 'english']

# Create a mask where each review has more than one word and at least one alphabetic character
mask = df_steamAPI['review'].apply(lambda x: len(re.findall(r'\b\w+\b', str(x))) > 5 and bool(re.search('[a-zA-Z]', str(x))))

# Apply the mask to the DataFrame to filter out reviews
df_steamAPI = df_steamAPI[mask]

In [32]:
# More than 100k reviews dropped (non-english or non-usable review text)
df_steamAPI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208424 entries, 5 to 338567
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      208424 non-null  int64  
 1   query_summary                   0 non-null       float64
 2   cursors                         0 non-null       float64
 3   recommendationid                208424 non-null  float64
 4   language                        208424 non-null  object 
 5   review                          208424 non-null  object 
 6   timestamp_created               208424 non-null  float64
 7   timestamp_updated               208424 non-null  float64
 8   voted_up                        208424 non-null  object 
 9   votes_up                        208424 non-null  float64
 10  votes_funny                     208424 non-null  float64
 11  weighted_vote_score             208424 non-null  float64
 12  comment_count        

In [34]:
# Take only review and recommendation/label columns
df_steamAPI['Texts'] = df_steamAPI.iloc[:, 5]
df_steamAPI['Label'] = df_steamAPI.iloc[:, 8]
df_steamAPI = df_steamAPI[['Texts', 'Label']]

df_kaggle['Texts'] = df_kaggle.iloc[:, 6]
df_kaggle['Label'] = df_kaggle.iloc[:, 5]
df_kaggle = df_kaggle[['Texts', 'Label']]

In [41]:
df_steamAPI.Label.unique()

array([1, 0])

In [42]:
df_kaggle.Label.unique()

array([1, 0], dtype=int64)

In [40]:
# Replace values in 'Label' columns so that True or 'Recommended' becomes 1 and false or 'Not recommended' becomes 0

df_kaggle['Label'] = df_kaggle['Label'].replace({'Recommended': 1, 'Not Recommended': 0})

# Convert boolean to integer (True -> 1, False -> 0)
df_steamAPI['Label'] = df_steamAPI['Label'].astype(int)

C:\Users\brynj\AppData\Local\Temp\ipykernel_10664\3139352698.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_kaggle['Label'] = df_kaggle['Label'].replace({'Recommended': 1, 'Not Recommended': 0})
C:\Users\brynj\AppData\Local\Temp\ipykernel_10664\3139352698.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steamAPI['Label'] = df_steamAPI['Label'].astype(int)


In [47]:
df_steamAPI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208424 entries, 5 to 338567
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Texts   208424 non-null  object
 1   Label   208424 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 4.0+ MB


In [48]:
df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 433375 entries, 0 to 434890
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Texts   433375 non-null  object
 1   Label   433375 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.9+ MB


In [50]:
# Concatenate the dataframes
concatenated_df = pd.concat([df_steamAPI, df_kaggle], ignore_index=True)

In [52]:
concatenated_df.head()

,Texts,Label
0,Came in expecting a mid game. Come out being i...,1
1,Would be nice if there where more maps,1
2,i love these flamey lil guys :3,1
3,this game is so good it is hard but fun at the...,1
4,"I got this game free at PAX 2024, not only did...",1


In [55]:
# Save to csv file
concatenated_df.to_csv('total_reviews_text_and_label.csv')